In [99]:
using Printf
m = 3
n = 4
A = rand(0:3, (m,n))
b = rand(0:3,m)
print(A)
println()
print(b)

[1 0 1; 0 0 0; 2 2 1]
[1, 3, 0]

### Idea
---
$$
\begin{gather}
    A' = \begin{bmatrix} A &  b \end{bmatrix} \\
    U' = CA' = \begin{bmatrix} CA &  Cb \end{bmatrix} \\
    U' = \begin{bmatrix} U &  Cb \end{bmatrix} \\
\end{gather}
$$
If $Cb$ is a pivot column, then the system is unsolvable, else it is consistent and has one or more solutions.

In [100]:
function rankconsistency(A::Matrix{Int64}, b::Vector{Int64})
    consistent::Bool = false
    U = rowreduce(A)
    return Bool,U  
end 


rankconsistency (generic function with 1 method)

In [101]:
Add_matrix = [  0 1 2 3  ;  1  0  3  2 ; 2 3 0 1 ; 3 2 1 0 ]
function AddGf(i, j)
    return Add_matrix[i-1][j-1]
end

AddGf (generic function with 1 method)

In [102]:
Mul_matrix = [ 0 0 0 0 ; 0  1  2  3 ; 0  2  3  1 ; 0 3 1  2 ]
function MulGf(i, j)
    return Mul_matrix[i-1][j-1]
end

MulGf (generic function with 1 method)

In [103]:
Inv_vec = [1, 3, 2]

3-element Vector{Int64}:
 1
 3
 2

In [104]:
function RowReduction(A::Matrix{Int64})

end

RowReduction (generic function with 1 method)

In [105]:
function PivotSubtract(A, pivot_row)
    for i in pivot_row + 1:size(A, 1)
        RowSubtract(A,pivot_row, i)
    end 
end

PivotSubtract (generic function with 2 methods)

In [106]:
function RowSubtract(A, i, j)
    @printf("Subtracting Row %d form Row %d \n", i , j)
end

RowSubtract (generic function with 1 method)

In [107]:
PivotSubtract(A, 1)

Subtracting Row 1 form Row 2 
Subtracting Row 1 form Row 3 
